# Imports

In [1]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

/home/server1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/server1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/server1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/server1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

# Variables

In [26]:
data_path="../../PlantVillage/"
image_size=tuple((150,150))
batch_size=256
#epochs=10

In [3]:
!ls {data_path}

Pepper__bell___Bacterial_spot  Tomato_Late_blight
Pepper__bell___healthy	       Tomato_Leaf_Mold
Potato___Early_blight	       Tomato_Septoria_leaf_spot
Potato___healthy	       Tomato_Spider_mites_Two_spotted_spider_mite
Potato___Late_blight	       Tomato__Target_Spot
Tomato_Bacterial_spot	       Tomato__Tomato_mosaic_virus
Tomato_Early_blight	       Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato_healthy


# Convert image to array and resize it

In [4]:
def convert_resize_image(image):
    try:
        image=cv2.imread(image)
        image=cv2.resize(image, image_size)
        return img_to_array(image)
    except Exception as e:
        # print(e, image)
        return None

# Read images from folders

In [5]:
images=[]
labels=[]

plant_disease_folders = os.listdir(f"{data_path}")

for plant_disease_folder in plant_disease_folders:
    plant_disease_images = os.listdir(f"{data_path}/{plant_disease_folder}/")
    
    for image in plant_disease_images:
        image_dir = f"{data_path}/{plant_disease_folder}/{image}"
        img = convert_resize_image(image_dir)
        if img is not None:
            images.append(convert_resize_image(image_dir))
            labels.append(plant_disease_folder)
print(images)
 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Encoding the labels

In [92]:
label_encoder=LabelEncoder()
label_binarizer=LabelBinarizer()

image_labels_enc=label_encoder.fit_transform(labels)
image_labels_bin=label_binarizer.fit_transform(labels)

In [8]:
labels

['Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell

In [9]:
image_labels_enc

array([ 0,  0,  0, ..., 12, 12, 12])

In [10]:
image_labels_bin[11]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
label_binarizer.classes_

array(['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
       'Potato___Early_blight', 'Potato___Late_blight',
       'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
       'Tomato_Late_blight', 'Tomato_Leaf_Mold',
       'Tomato_Septoria_leaf_spot',
       'Tomato_Spider_mites_Two_spotted_spider_mite',
       'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
       'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'], dtype='<U43')

# Normalize the images

In [12]:
normalized_images=np.array(images) /255.0

In [13]:
normalized_images

array([[[[0.07843138, 0.0627451 , 0.08627451],
         [0.14901961, 0.13333334, 0.15686275],
         [0.2509804 , 0.22352941, 0.2509804 ],
         ...,
         [0.57254905, 0.48235294, 0.5019608 ],
         [0.5647059 , 0.4745098 , 0.49411765],
         [0.5529412 , 0.46666667, 0.4862745 ]],

        [[0.43137255, 0.40392157, 0.42745098],
         [0.5647059 , 0.5372549 , 0.5647059 ],
         [0.67058825, 0.6392157 , 0.6666667 ],
         ...,
         [0.57254905, 0.48235294, 0.5019608 ],
         [0.56078434, 0.47058824, 0.49019608],
         [0.54509807, 0.45490196, 0.4745098 ]],

        [[0.7058824 , 0.6627451 , 0.69411767],
         [0.74509805, 0.7019608 , 0.73333335],
         [0.7529412 , 0.7058824 , 0.73333335],
         ...,
         [0.5647059 , 0.4745098 , 0.49411765],
         [0.54901963, 0.45882353, 0.47843137],
         [0.5372549 , 0.44313726, 0.4627451 ]],

        ...,

        [[0.5294118 , 0.45490196, 0.49019608],
         [0.5882353 , 0.5137255 , 0.54901963]

In [14]:
normalized_images.mean()

0.44788694

# Train/test split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(normalized_images, image_labels_bin, test_size=0.3, random_state=42)

# Image Transformations

In [16]:
train_datagen = ImageDataGenerator(
    rotation_range=25, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

# Build Models

In [17]:
if K.image_data_format() == 'channels_first': 
    input_shape = (3,) + image_size 
    batch_norm_axis=1
else: 
    input_shape = image_size + (3,)
    batch_norm_axis=-1

In [99]:
def add_cnn_unit(model, filters=32, pool_size=(3, 3), input_layer=True, kernel_initializer='glorot_uniform'):
    if input_layer:
        model.add(Conv2D(filters, (3, 3), padding="same", input_shape=input_shape, kernel_initializer=kernel_initializer))
    else:
        model.add(Conv2D(filters, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    
    model.add(Activation('relu')) 
    model.add(BatchNormalization(axis=batch_norm_axis))
    model.add(MaxPooling2D(pool_size=pool_size)) 
    
def get_model(kernel_initializer='glorot_uniform'):
    model = Sequential()
    add_cnn_unit(model, filters=32, pool_size=(3, 3), input_layer=True, kernel_initializer=kernel_initializer)
    add_cnn_unit(model, filters=64, pool_size=(2, 2), input_layer=True, kernel_initializer=kernel_initializer)
    add_cnn_unit(model, filters=128, pool_size=(2, 2), input_layer=True, kernel_initializer=kernel_initializer)
    model.add(Dropout(0.25))
    model.add(Flatten()) 
    model.add(Dense(1000, kernel_initializer=kernel_initializer)) 
    model.add(Activation('relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(len(label_binarizer.classes_), kernel_initializer=kernel_initializer)) 
    model.add(Activation('softmax')) 
    return model

# glorot_uniform init

In [103]:
!mkdir model_7
model = get_model(kernel_initializer='glorot_uniform')
optimizer=Adam(lr=0.003, decay=0.001/ 3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
callbacks = [
    ModelCheckpoint("./model_7/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor="val_loss"),
    EarlyStopping(monitor="val_acc", patience=10),
    ReduceLROnPlateau(min_lr=0.00001, patience=3, verbose=1, factor=0.2)
]

In [43]:
fitter=model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size), 
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=2000, 
    verbose=1,
    callbacks=callbacks
) 

Epoch 1/2000
56/56 [==============================] - 37s 658ms/step - loss: 0.5967 - acc: 0.9268 - val_loss: 1.9462 - val_acc: 0.8735
Epoch 2/2000
56/56 [==============================] - 36s 637ms/step - loss: 0.1140 - acc: 0.9606 - val_loss: 1.7072 - val_acc: 0.8802
Epoch 3/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0878 - acc: 0.9685 - val_loss: 1.4023 - val_acc: 0.8802
Epoch 4/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0734 - acc: 0.9736 - val_loss: 1.5268 - val_acc: 0.8805
Epoch 5/2000
56/56 [==============================] - 35s 634ms/step - loss: 0.0635 - acc: 0.9774 - val_loss: 0.9992 - val_acc: 0.8870
Epoch 6/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0579 - acc: 0.9792 - val_loss: 0.9646 - val_acc: 0.8887
Epoch 7/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0547 - acc: 0.9802 - val_loss: 0.9368 - val_acc: 0.8972
Epoch 8/2000
56/56 [==============================] - 3

# he init

In [46]:
!mkdir model_8
model = get_model(kernel_initializer='he_uniform')
optimizer=Adam(lr=0.003, decay=0.001/ 3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
callbacks = [
    ModelCheckpoint("./model_8/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor="val_loss"),
    EarlyStopping(monitor="val_acc", patience=10),
    ReduceLROnPlateau(min_lr=0.00001, patience=3, verbose=1, factor=0.2)
]
fitter=model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size), 
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=2000, 
    verbose=1,
    callbacks=callbacks
)    

Epoch 1/2000
56/56 [==============================] - 37s 656ms/step - loss: 0.9803 - acc: 0.9102 - val_loss: 1.0941 - val_acc: 0.8796
Epoch 2/2000
56/56 [==============================] - 36s 636ms/step - loss: 0.1372 - acc: 0.9545 - val_loss: 0.8290 - val_acc: 0.8921
Epoch 3/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0941 - acc: 0.9653 - val_loss: 1.0567 - val_acc: 0.8950
Epoch 4/2000
56/56 [==============================] - 35s 634ms/step - loss: 0.0789 - acc: 0.9709 - val_loss: 1.1730 - val_acc: 0.8939
Epoch 5/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0702 - acc: 0.9742 - val_loss: 0.7901 - val_acc: 0.9015
Epoch 6/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0611 - acc: 0.9771 - val_loss: 0.5182 - val_acc: 0.9207
Epoch 7/2000
56/56 [==============================] - 35s 632ms/step - loss: 0.0528 - acc: 0.9808 - val_loss: 0.5053 - val_acc: 0.9255
Epoch 8/2000
56/56 [==============================] - 3

# resnet 50

In [91]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

def build_resnet50(img_shape=(3, 224, 224), n_classes=1000, l2_reg=0.,
                load_pretrained=False, freeze_layers_from='base_model'):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = 'imagenet'
    else:
        weights = None

    # Get base model
    base_model = ResNet50(include_top=False, weights=weights,
                       input_tensor=None, input_shape=img_shape)

    # Add final layers
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(n_classes, activation='softmax', name='fc1000')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == 'base_model':
            print ('   Freezing base model layers')
            for layer in base_model.layers:
                layer.trainable = False
        else:
            print ('   Freezing from layer 0 to ' + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
                layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
                layer.trainable = True

    return model 

!mkdir model_11
model = build_resnet50(
    img_shape=input_shape, 
    n_classes=len(label_binarizer.classes_),
    load_pretrained=True,
)
optimizer=Adam(lr=0.003, decay=0.001/ 3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
callbacks = [
    ModelCheckpoint("./model_11/weights_first.hdf5", monitor="val_loss", save_best_only=True),
    ReduceLROnPlateau(min_lr=0.00001, patience=3, verbose=1, factor=0.2)
]
fitter=model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size), 
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=20, 
    verbose=1,
    callbacks=callbacks
)    

model = build_resnet50(
    img_shape=input_shape, 
    n_classes=len(label_binarizer.classes_),
    load_pretrained=False,
    freeze_layers_from=0
)
model.load_weights("./model_11/weights_first.hdf5")
optimizer=Adam(lr=0.003, decay=0.001/ 3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
callbacks = [
    ModelCheckpoint("./model_11/weights.{epoch:02d}.hdf5", monitor="val_loss"),
    ReduceLROnPlateau(min_lr=0.00001, patience=3, verbose=1, factor=0.2)
]
fitter=model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=4), 
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // 4,
    epochs=20, 
    verbose=1,
    callbacks=callbacks
)    

/home/server1/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


   Freezing base model layers
Epoch 1/20
56/56 [==============================] - 128s 2s/step - loss: 0.6058 - acc: 0.9305 - val_loss: 0.6216 - val_acc: 0.8809
Epoch 2/20
56/56 [==============================] - 39s 691ms/step - loss: 0.5273 - acc: 0.9412 - val_loss: 0.6903 - val_acc: 0.8802
Epoch 3/20
56/56 [==============================] - 38s 677ms/step - loss: 0.5370 - acc: 0.9412 - val_loss: 0.6609 - val_acc: 0.8814
Epoch 4/20
55/56 [============================>.] - ETA: 0s - loss: 0.4831 - acc: 0.9476
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
56/56 [==============================] - 41s 724ms/step - loss: 0.4819 - acc: 0.9477 - val_loss: 0.6333 - val_acc: 0.8825
Epoch 5/20
56/56 [==============================] - 37s 665ms/step - loss: 0.4401 - acc: 0.9522 - val_loss: 0.6428 - val_acc: 0.8853
Epoch 6/20
56/56 [==============================] - 37s 658ms/step - loss: 0.4283 - acc: 0.9530 - val_loss: 0.6339 - val_acc: 0.8876
Epoch 7/20
55/5

# Load Model / Test model

In [82]:
from tensorflow.keras.models import load_model

def predict(model, X):
    preds = model.predict(X)
    return preds.argmax(axis=1)

def predict_one(model, x):
    return predict(model, x[None, ...])

def transcribe_preds(preds):
    return list(map(lambda x: label_binarizer.classes_[x], preds))

In [83]:
test_model = load_model('./model_7/weights.21-0.10.hdf5')

In [84]:
test_model.predict(X_test)

array([[8.6905307e-04, 7.6481271e-01, 2.9001659e-04, ..., 5.4014893e-03,
        6.7399251e-03, 4.9626380e-02],
       [0.0000000e+00, 0.0000000e+00, 6.0654715e-30, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [2.9313835e-05, 1.6257733e-05, 4.7063944e-09, ..., 4.2781062e-06,
        2.6795174e-12, 3.2369284e-11],
       ...,
       [9.4248449e-09, 7.5342811e-08, 4.1893482e-16, ..., 4.9039093e-13,
        8.4591341e-01, 1.5146168e-05],
       [1.4101957e-09, 1.0078281e-11, 2.0283273e-06, ..., 4.8173249e-10,
        3.3327873e-13, 2.9475897e-12],
       [3.4809518e-06, 9.9992955e-01, 5.7787474e-07, ..., 7.5698371e-12,
        1.2460416e-13, 3.9284841e-06]], dtype=float32)

In [85]:
transcribe_preds(predict(test_model, X_test)), transcribe_preds(predict_one(test_model, X_test[0]))

(['Pepper__bell___healthy',
  'Tomato_healthy',
  'Tomato_Late_blight',
  'Tomato_Spider_mites_Two_spotted_spider_mite',
  'Tomato_Leaf_Mold',
  'Tomato_Late_blight',
  'Tomato_Bacterial_spot',
  'Tomato_Septoria_leaf_spot',
  'Tomato_Late_blight',
  'Tomato__Tomato_YellowLeaf__Curl_Virus',
  'Tomato_Bacterial_spot',
  'Tomato__Tomato_YellowLeaf__Curl_Virus',
  'Tomato_Septoria_leaf_spot',
  'Tomato_Bacterial_spot',
  'Tomato_Late_blight',
  'Tomato_healthy',
  'Pepper__bell___Bacterial_spot',
  'Tomato_Septoria_leaf_spot',
  'Tomato_Spider_mites_Two_spotted_spider_mite',
  'Tomato_healthy',
  'Tomato__Tomato_YellowLeaf__Curl_Virus',
  'Tomato_Septoria_leaf_spot',
  'Tomato_healthy',
  'Potato___Early_blight',
  'Potato___healthy',
  'Pepper__bell___healthy',
  'Tomato__Tomato_YellowLeaf__Curl_Virus',
  'Tomato_healthy',
  'Tomato_Early_blight',
  'Pepper__bell___Bacterial_spot',
  'Pepper__bell___Bacterial_spot',
  'Pepper__bell___healthy',
  'Tomato_Septoria_leaf_spot',
  'Tomato__To

In [53]:
len(X_test)

6192

In [61]:
label_binarizer.classes_

array(['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
       'Potato___Early_blight', 'Potato___Late_blight',
       'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
       'Tomato_Late_blight', 'Tomato_Leaf_Mold',
       'Tomato_Septoria_leaf_spot',
       'Tomato_Spider_mites_Two_spotted_spider_mite',
       'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
       'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'], dtype='<U43')

In [62]:
preds = np.array([0, 1])

In [66]:
label_binarizer.classes_

array(['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
       'Potato___Early_blight', 'Potato___Late_blight',
       'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
       'Tomato_Late_blight', 'Tomato_Leaf_Mold',
       'Tomato_Septoria_leaf_spot',
       'Tomato_Spider_mites_Two_spotted_spider_mite',
       'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
       'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'], dtype='<U43')

In [72]:
list(map(lambda x: label_binarizer.classes_[x], preds))

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']

In [70]:
print(x)